The goal of this script is to process the data for data visulization challenge. For the visualization, I will develop a shiny app that displays the following information:

- for each department
- for male and female
- for highest rating and lowest rating

**Display the 50 most relavent words in wordcloud.**


Since the shiny app can't load too much data, I will preprocess the data here and save the compact information for later.

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
train = pd.read_csv("../Data/train.csv")

In [4]:
# later probably will drop some columns
print train.columns

Index([u'id', u'tid', u'dept', u'date', u'forcredit', u'attendance',
       u'textbookuse', u'interest', u'grade', u'tags', u'comments',
       u'helpcount', u'nothelpcount', u'online', u'profgender', u'profhotness',
       u'helpfulness', u'clarity', u'easiness', u'quality'],
      dtype='object')


Now let me take a look at `comments` column. 

In [5]:
# Now let's take a look at the `comments` column
stop_words = './stop_words.txt'
with open(stop_words, 'r') as f:
    words = f.read()
    words = words.split('\n')[:-1]
# Create a CountVectorizer object and use the fit_transform method to learn the document-term matrix.
count_vec = CountVectorizer(min_df=120, ngram_range=(1,2), stop_words=words)
doc_word_matrix = count_vec.fit_transform(train['comments'].fillna(''))

In [6]:
# DataFrame objects can be created from dictionaries.  We start by initializing a dictionary
# that has just the ids corresponding to comments.
freq_dict = {'id':train.id}
count = 0

# The count_vec object stores the words from the text as a list.  Word i corresponds to index i.
for word in count_vec.get_feature_names():
    # Pull a column from the document-term matrix and assign it to its corresponding word
    # in the dictionary.
    freq_dict[word] = doc_word_matrix[:,count].toarray().T[0]  # I admit it, this looks hacky and gross.
    count+=1
    
# Create a new DataFrame object with the dictionary we created.  This will be much easier to search
# and analyze later.
doc_word_df = pd.DataFrame(data=freq_dict)

Now, I have the original data and processed comments ready.

In [7]:
# Find duplicate rows.  Based on my observations, a duplicate of a row always immediately follows the row of which it 
# is a copy.  Thus, we will go row by row and see if any two subsequent ids are the same.  If so, we will add the 
# index of the id to a list for future removal.
duplicates = []
for i in range(len(doc_word_df)-1):
    if doc_word_df['id'][i] == doc_word_df['id'][i+1]:
        duplicates.append(i)

In [8]:
# drop the duplciates
train.drop(duplicates, inplace = True)
doc_word_df.drop(duplicates, inplace = True)

In [9]:
for each_dept in train.dept.unique():
    print each_dept, ':', train.ix[train.dept == each_dept].shape[0]

Business : 4200
Economics : 4926
Religion : 9417
Church History : 390
Social Science : 850
Anthropology : 1563
Accounting : 1764
Music : 1446
Psychology : 5012
Chemistry : 9343
Theater : 895
Biology : 7989
Fine Arts : 964
History : 4817
Political Science : 4506
Science : 2743
Interior Design : 104
Humanities : 1788
Mathematics : 9391
Statistics : 679
Art : 499
Physiology : 149
Agriculture : 413
Geography : 1387
Spanish : 534
Physics : 3106
Sociology : 3308
Family & Consumer Science : 175
Writing : 316
Film : 244
Philosophy : 1444
Communication : 2054
Computer Science : 2007
Nutrition & Food Science : 95
English : 6637
Finance : 398
Japanese : 91
Geology : 1071
Engineering : 2530
Marriage Family & Human Dev : 49
Languages : 2673
Physical Sciences : 129
Dance : 253
Student Services : 150
French : 95
Health Science : 902
Classical Studies : 37
Literature : 797
English Language & Literature : 61
Information Systems : 280
Mathematics Education : 60
Classics : 537
Exercise & Sport Science : 

Above results shows that we need to group different departments together to have enough sample to do regression for each department. Next, we are going to output the department in the train.csv and group them according to the one we see on that New Yorker website.

In [10]:
fout = open('all_depts.csv', 'w')
for each_dept in train.dept.unique():
    fout.write(each_dept + ',' + str(train.ix[train.dept == each_dept].shape[0]) + '\n')
fout.close()

I manually divide above departments in to the following categories, which I saved in `grouped depts.txt`. Next I will read the file and add a new columns that map the current departments to divided departments.

In [11]:
# we notice that there is a department called 'Gender, Women, and Sex Studies'. For later to separate each department in a
# group, we need use ',' to split the row string. So, we are going to change it to 'Gender_Women_and Sex Studies'

train.ix[train.dept == 'Gender, Women, and Sex Studies', 'dept'] = 'Gender_Women_and Sex Studies'
train.ix[train.dept =='Gender_Women_and Sex Studies']

,id,tid,dept,date,forcredit,attendance,textbookuse,interest,grade,tags,comments,helpcount,nothelpcount,online,profgender,profhotness,helpfulness,clarity,easiness,quality
83444,24317131,1522784,Gender_Women_and Sex Studies,01/21/2015,Yes,Not Mandatory,Essential to passing,It's my life,A,"[""Would take again"", ""Inspirational"", ""Gives g...",Prof Isoke is incredible Taking class from her...,0,0,NaN,0,0,5,4,3,9
83445,24296609,1522784,Gender_Women_and Sex Studies,01/16/2015,Yes,NaN,You need it sometimes,Really into it,NaN,"[""Inspirational"", ""Participation matters"", ""Th...",Lots of individualized attention in office hou...,0,0,NaN,0,0,4,3,2,7
83446,23995183,1522784,Gender_Women_and Sex Studies,12/09/2014,Yes,Not Mandatory,You need it sometimes,Low,NaN,"[""Better Like Group Projects"", ""Get ready to r...",Professor Isoke is very unorganized and change...,0,0,NaN,0,0,2,2,3,4
83447,23519233,1522784,Gender_Women_and Sex Studies,08/11/2014,NaN,NaN,You need it sometimes,Sorta interested,NaN,[],DO NOT TAKE THIS CLASS She is very unorganized...,0,0,NaN,0,0,1,1,3,2
83448,23341363,1522784,Gender_Women_and Sex Studies,06/03/2014,NaN,NaN,Barely cracked it open,It's my life,NaN,[],Zenzele is very hard to get ahold of often ref...,0,0,NaN,0,0,2,2,4,4
83449,23308365,1522784,Gender_Women_and Sex Studies,05/24/2014,NaN,Not Mandatory,Essential to passing,It's my life,NaN,[],Great,0,0,NaN,0,0,5,5,5,10
83450,23257620,1522784,Gender_Women_and Sex Studies,05/15/2014,NaN,NaN,You need it sometimes,Low,NaN,[],NEVER TAKE THIS CLASS While I was very interes...,0,0,NaN,0,0,1,1,3,2
83451,22989872,1522784,Gender_Women_and Sex Studies,04/04/2014,NaN,Mandatory,Essential to passing,Really into it,NaN,[],Its an easy class Getting As on papers and tes...,0,0,NaN,0,0,3,2,5,5
83452,21620994,1522784,Gender_Women_and Sex Studies,04/28/2013,NaN,NaN,Essential to passing,Really into it,NaN,[],While I feel like this class is very repetitiv...,0,0,NaN,0,0,5,4,5,9
83453,21527884,1522784,Gender_Women_and Sex Studies,04/12/2013,NaN,NaN,Essential to passing,Low,NaN,[],Isoke knows her stuff She knows the field and ...,0,1,NaN,0,0,5,5,3,10


In [13]:
# create a department mapping dictionary
dept_mapping = {}
f = open("grouped depts.txt")
for line in f:
    temp_div = line.split(': ')
    dept_group = temp_div[0]
    temp_depts = temp_div[1].strip('\n')
    for temp_dept in temp_depts.split(', '):
        dept_mapping[temp_dept] = dept_group
f.close()
print dept_mapping

{'': 'Accounting', 'Education & Human Ecology': 'Humanities', 'Business Administration': 'Business', 'Biochemistry': 'Biology', 'Sign Language': 'Communication', 'Human Resources': 'Business', 'Microbiology': 'Biology', 'Physical Sciences': 'Science', 'Religion': 'Religion', 'Family Social Science': 'Sociology', 'Engineering Graphics Tech.': 'Engineering', 'Business': 'Business', 'Fine Arts': 'Arts', 'Consumer Science': 'Business', 'Library Science': 'Science', 'Environmental Studies': 'Engineering', 'French': 'Languages', 'Anthropology': 'Anthropology', 'Marriage Family & Human Dev': 'Sociology', 'Exercise & Sport Science': 'Science', 'Computer Science': 'Computer Science', 'Life Science': 'Biology', 'Physical Education': 'Education', 'Writing Studies': 'Communication', 'Public Health': 'Health Science', 'Curriculum & Instruction': 'Education', 'Family & Consumer Science': 'Sociology', 'Theatre Arts & Dance': 'Arts', 'Health & Physical Education': 'Education', 'Kinesiology': 'Health S

In [14]:
## add a another columns in train `dept_group` 
train['dept_group'] = train['dept'].map(dept_mapping)

In [15]:
## Now let's see how many sample each dept_group contains
for each_dept in train.dept_group.unique():
    print each_dept, ':', train.ix[train.dept_group == each_dept].shape[0]

Business : 5381
Economics : 4938
Religion : 10414
Sociology : 5910
Anthropology : 1953
Accounting : 1764
Music : 1446
Psychology : 5042
Chemistry : 9343
Arts : 3978
Biology : 8678
History : 4822
Political Science : 4856
Science : 5827
Humanities : 1829
Mathmatics : 10070
Agriculture and Food Sciences : 610
Languages : 4209
Physics : 3509
Communication : 3614
nan : 0
Computer Science : 2465
English : 6637
Engineering : 3295
Education : 1322
Health Science : 1544
Literature : 1778
Other : 632
Law : 449


From above result, we can see that the sample for each category is still too little, compared to 2000+ features (words). In this case, we would like to further group the departmenets together

In [16]:
# use 'group depts v2.txt' to  create a department mapping dictionary
dept_mapping = {}
f = open("group depts v2.txt")
for line in f:
    temp_div = line.split(': ')
    dept_group = temp_div[0]
    temp_depts = temp_div[1].strip('\n')
    for temp_dept in temp_depts.split(', '):
        dept_mapping[temp_dept] = dept_group
f.close()

In [17]:
## add a another columns in train `dept_group` 
train['dept_group'] = train['dept'].map(dept_mapping)

In [18]:
## Now let's see how many sample each dept_group contains
for each_dept in train.dept_group.unique():
    print each_dept, ':', train.ix[train.dept_group == each_dept].shape[0]

Business&Economics&Accounting : 12532
Religion&Philosophy : 11858
Humanity&Sociology&Psychology : 14734
Literature&Art&Music : 31340
Science : 18679
Bio&Health : 10222
Mathmatics : 10070
Other : 2564
Engineering : 5760
nan : 0


In [18]:
temp_word_df = doc_word_df[train.dept_group == 'Engineering']

In [20]:
print temp_word_df.columns[temp_word_df.sum()==2]
del temp_word_df

Index([u'121', u'75', u'accounting', u'adorable', u'always interesting',
       u'appreciated', u'articles', u'awful teacher', u'beginning semester',
       u'best math', u'biased', u'bomb', u'book just', u'boring tests',
       u'buying', u'church', u'clicker', u'cumulative', u'daily quizzes',
       u'drops', u'easy pay', u'eyes', u'favorite semester', u'fill',
       u'french', u'fun listen', u'fun really', u'get just', u'get little',
       u'go get', u'go read', u'go ta', u'good ta', u'grammar',
       u'however tests', u'imagine', u'interesting easy',
       u'interesting listen', u'interesting take', u'just pay', u'just want',
       u'keeps things', u'know exactly', u'know lot', u'know students',
       u'know study', u'knows much', u'lecture interesting', u'lecture take',
       u'lectures always', u'lectures entertaining', u'lectures long',
       u'level course', u'liberal', u'lot good', u'lot really',
       u'lots reading', u'loved brother', u'loved professor', u'loving',


In [19]:
## drop 'comments' column from train data to save some space
print train.columns
train.drop(['comments', 'date', 'tags', 'textbookuse', 'attendance', 'date', 'forcredit'], axis = 1, inplace = True)
print 'after dropping: \n', train.columns

Index([u'id', u'tid', u'dept', u'date', u'forcredit', u'attendance',
       u'textbookuse', u'interest', u'grade', u'tags', u'comments',
       u'helpcount', u'nothelpcount', u'online', u'profgender', u'profhotness',
       u'helpfulness', u'clarity', u'easiness', u'quality', u'dept_group'],
      dtype='object')
after dropping: 
Index([u'id', u'tid', u'dept', u'interest', u'grade', u'helpcount',
       u'nothelpcount', u'online', u'profgender', u'profhotness',
       u'helpfulness', u'clarity', u'easiness', u'quality', u'dept_group'],
      dtype='object')


### Lasso regression to find the most relavent words for best and worst ratings

The reason to use lasso regression is that I think the input data is highly sparse.

First, let me test with `Bio&Health` data.

In [61]:
temp_train = train.ix[train.dept_group == 'Other']
temp_word_df = doc_word_df[train.dept_group == 'Other']
print temp_train.shape, temp_word_df.shape

(2564, 15) (2564, 2849)


In [62]:
## For temp_word_df, there must be columns that is all-zero, remove those columns
columns = temp_word_df.columns[temp_word_df.sum() == 0]
print columns
temp_word_df.drop(columns, axis = 1, inplace = True)
print temp_word_df.shape

Index([u'103', u'105', u'121', u'202', u'80', u'95', u'absolutely love',
       u'accounting', u'almost impossible', u'amount reading',
       ...
       u'tests multiple', u'tests straight', u'tests straightforward',
       u'texts', u'throws', u'time tests', u'unrelated', u'uplifting',
       u'went lecture', u'went office'],
      dtype='object', length=140)
(2564, 2709)


c:\Users\Xinyu\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
## import several modules
from sklearn import linear_model, cross_validation
from sklearn.metrics import mean_squared_error

In [63]:
## separate data for training and cross validating

Ytrain = np.ravel(temp_train.quality)
Xtr, Xval, Ytr, Yval = cross_validation.train_test_split(np.array(temp_word_df), Ytrain, test_size = 0.25, random_state = 0)

In [79]:
alpha_values = [0.001, 0.01]

mseTr = np.zeros((len(alpha_values),))
mseVal = np.zeros((len(alpha_values),))

for i, alpha_value in enumerate(alpha_values):
    print 'alpha: ', alpha_value
    m = linear_model.Lasso(alpha = alpha_value)
    m.fit(Xtr, Ytr)
    YhatTr = m.predict(Xtr)
    YhatVal = m.predict(Xval)
    mseTr[i] = mean_squared_error(YhatTr, Ytr)
    mseVal[i] = mean_squared_error(YhatVal, Yval)
    print mseTr[i], mseVal[i]

alpha:  0.001
1.30580795341 4.80455002368
alpha:  0.01
3.57077026713 4.01357975782


In [93]:
# Next, let's systematically cross validate for each dept_group
mseTr = {}
mseVal = {}

alpha_values = [0.000001, 0.00001, 0.0001, 0.001, 0.1, 1, 10]

for each_dept in train.dept_group.unique():
    if train.ix[train.dept_group == each_dept].shape[0] != 0:
        mseTr[each_dept] = {}
        mseVal[each_dept] = {}
        Xtrain = doc_word_df.ix[train.dept_group == each_dept] # need to drop all-zero columns
        columns = Xtrain.columns[Xtrain.sum() == 0]
        # divide data set
        Xtrain = Xtrain.drop(columns, axis = 1, inplace = False)
        Ytrain = np.ravel(train.ix[train.dept_group == each_dept, 'quality'])
        Xtr, Xval, Ytr, Yval = cross_validation.train_test_split(np.array(Xtrain), 
                                                                 Ytrain, 
                                                                 test_size = 0.25, 
                                                                 random_state = 0)
        # Cross validation - tune alpha value
        print 'Department: ', each_dept
        for i, alpha_value in enumerate(alpha_values):
            print '\t alpha: ', alpha_value
            m = linear_model.Lasso(alpha = alpha_value)
            m.fit(Xtr, Ytr)
            YhatTr = m.predict(Xtr)
            YhatVal = m.predict(Xval)
            mseTr_v = mean_squared_error(YhatTr, Ytr)
            mseVal_v = mean_squared_error(YhatVal, Yval)
            mseTr[each_dept][alpha_value] = mseTr_v
            mseVal[each_dept][alpha_value] = mseVal_v
            print '\t\t', mseTr_v, mseVal_v

Department:  Business&Economics&Accounting
	 alpha:  1e-06
		2.46446910251 4.93032209964
	 alpha:  1e-05
		2.46488807812 4.87885160351
	 alpha:  0.0001
		2.4969230673 4.48118901868
	 alpha:  0.001
		3.08923882695 3.79616548449
	 alpha:  0.1
		6.39674666917 6.46675850262
	 alpha:  1
		7.29232824897 7.41598780283
	 alpha:  10
		7.29232824897 7.41598785656
Department:  Religion&Philosophy
	 alpha:  1e-06
		1.82878012557 3.80631720274
	 alpha:  1e-05
		1.82931066631 3.7414872034
	 alpha:  0.0001
		1.8658729671 3.31070863167
	 alpha:  0.001
		2.3725802407 2.8292489604
	 alpha:  0.1
		5.12237588789 4.88179639644
	 alpha:  1
		5.53787598214 5.2796645199
	 alpha:  10
		5.53787598214 5.27966455442
Department:  Humanity&Sociology&Psychology
	 alpha:  1e-06
		2.44111484169 4.45030459593
	 alpha:  1e-05
		2.44145724832 4.41171409928
	 alpha:  0.0001
		2.46697693492 4.11919015805
	 alpha:  0.001
		2.98254276835 3.59364556149
	 alpha:  0.1
		6.12433356669 6.43130467332
	 alpha:  1
		6.65914838201 6.

c:\Users\Xinyu\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:444: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


From above results, we notice that alpha = 0.001 works for all of the dept_group's. Therefore we can just fix alpha value to 0.001.

Next step, we will train the model on whole dataset with alpha = 0.001. After training the model, we are going to save 

In [50]:
for each_dept in train.dept_group.unique():
    if train.ix[train.dept_group == each_dept].shape[0] != 0:
        
        Xtrain = doc_word_df.ix[train.dept_group == each_dept] # need to drop all-zero columns
        columns = Xtrain.columns[Xtrain.sum() == 0]
        # divide data set
        Xtrain = Xtrain.drop(columns, axis = 1, inplace = False)
        Ytrain = np.ravel(train.ix[train.dept_group == each_dept, 'quality'])
        # Lasso regress
        m = linear_model.Lasso(alpha = 0.001)
        m.fit(np.array(Xtrain), Ytrain)
        # sort coefficients
        words_df = pd.DataFrame(data = {'words': Xtrain.columns, 
                                        'coef': m.coef_})
      
        words_df.sort('coef',ascending=False,inplace=True)
        print 'Department: ', each_dept
        print '\t\t Most Posivie: \n',  words_df[:10]
        print '\t\t Most Negative: \n', words_df[-10:]
        pos_words = words_df[:50]
        pos_words = pos_words.rename_axis({'coef': 'pos_coef', 'words': 'pos_words'}, axis = 1)
        neg_words = words_df[-50:]
        neg_words = neg_words.rename_axis({'coef': 'neg_coef', 'words': 'neg_words'}, axis = 1)
        saved_data  = pd.DataFrame(data = {
                'pos_coef': list(pos_words.pos_coef),
                'pos_words': list(pos_words.pos_words),
                'neg_coef': list(neg_words.neg_coef),
                'neg_words': list(neg_words.neg_words)
            })

        saved_data.to_csv(each_dept + '.csv', index = False)
        

Department:  Business&Economics&Accounting
		 Most Posivie: 
          coef         words
688   1.201208     excellent
758   1.099202     fantastic
1746  1.028272   outstanding
186   1.003040       awesome
588   0.981418     effective
91    0.963398       amazing
232   0.850389  best teacher
2122  0.838585         rocks
988   0.719587         great
223   0.699297          best
		 Most Negative: 
          coef             words
2484 -1.187698          terrible
192  -1.229235             awful
1831 -1.289786              poor
180  -1.328896             avoid
2655 -1.349817           useless
2642 -1.362923       unorganized
1183 -1.412557  incomprehensible
1148 -1.511784          horrible
2636 -1.610192         unhelpful
2794 -1.825893             worst
Department:  Religion&Philosophy
		 Most Posivie: 
          coef          words
1855  1.195875  pretty boring
730   0.825442       fabulous
185   0.769818        awesome
91    0.746406        amazing
679   0.656699      excellent
2739  0

c:\Users\Xinyu\Anaconda\lib\site-packages\IPython\kernel\__main__.py:16: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [49]:
pos_words.reset_index()
neg_words.reset_index()
pd.DataFrame(data = {'pos_coef': list(pos_words.pos_coef)})

,pos_coef
0,2.010465
1,1.543651
2,1.473251
3,1.259828
4,1.231161
5,1.213152
6,1.186783
7,1.125160
8,1.091822
9,1.090734


In [30]:
pos_words.rename_axis({'coef': 'c'}, axis = 1)

,c,words
417,-0.760902,costs
639,-0.767382,everyone else
1709,-0.774033,paid
2328,-0.796909,take unless
1214,-0.803235,jumps
2678,-0.813705,willing put
973,-0.832580,great sense
1044,-0.833749,hate
2568,-0.834151,unrelated
956,-0.844216,graduate
